In [1]:
cd ..

/Users/muhammadfarooq/Codebase/wood-log-detection/Pith2Bark


In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scikitplot as skplt

import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Dropout

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from src.cv_winner import winner_model_rings
from src.damage import merge_df
from src.damage import read_damage_min_max
from src.damage import read_damage_mean

# Damage Dataset

In [3]:
rmse_url = "cv_output/ringsranking_rmse.csv"
rings_url = "cv_output/ringsranking_rings.csv"
df_ringsranking = winner_model_rings(rmse_url, rings_url)

Polar Pith Canny(blur otsu) 12.829 (subseq)


In [4]:
rmse_url = "cv_output/pine_rmse.csv"
rings_url = "cv_output/pine_rings.csv"
df_pine = winner_model_rings(rmse_url, rings_url)

Polar Pith Canny(blur triangle) 19.888 (peaks)


In [5]:
rmse_url = "cv_output/fur_rmse.csv"
rings_url = "cv_output/fur_rings.csv"
df_fur = winner_model_rings(rmse_url, rings_url)

Polar Pith Canny(blur triangle) 13.429 (peaks)


In [6]:
rmse_url = "cv_output/tracy_rmse.csv"
rings_url = "cv_output/tracy_rings.csv"
df_tracy = winner_model_rings(rmse_url, rings_url)

Polar Canny(triangle) 39.684 (peaks)


In [7]:
rmse_url = "cv_output/data_rmse.csv"
rings_url = "cv_output/data_rings.csv"
df_data = winner_model_rings(rmse_url, rings_url)

Canny(triangle) 18.79 (subseq)


In [8]:
#damage
damage_url = "damage_dataset/damage.csv"

#merge
df_merge = merge_df(df_ringsranking, df_pine, df_fur, df_tracy, df_data)

df_damage_min_max = read_damage_min_max(damage_url, df_merge)

df_damage_mean = read_damage_mean(damage_url, df_merge)

In [9]:
#STOP

# Config

In [10]:
training_percent = 0.9

epochs, batch_size = 50, 32
kf = KFold(n_splits=3, shuffle=True, random_state=42)

# mode = max for monitro val_accuracy and val_prc
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=10,
    restore_best_weights=True)

METRICS = [
    tf.keras.metrics.TruePositives(name='tp'),
    tf.keras.metrics.FalsePositives(name='fp'),
    tf.keras.metrics.TrueNegatives(name='tn'),
    tf.keras.metrics.FalseNegatives(name='fn'), 
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
    tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

#columns to drop
columns = ['image', 'algo']

In [11]:
def heat_map_normalized(y_true, y_pred):
    
    class_names = ['Not Damage', 'Damaged']
    
    skplt.metrics.plot_confusion_matrix(y_true, y_pred,
                                        figsize=(4,3),
                                        normalize=True)
    
    # Customize axis tick labels
    plt.xticks([0,1], class_names)
    plt.yticks([0,1], class_names)
    
    plt.yticks(rotation=90)
    
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.show()

In [12]:
def heat_map(_cm):
    # plot confusion matrix as heatmap
    labels = ['Not Damage', 'Damaged']
    
    # Set up the matplotlib figure
    plt.figure(figsize=(4, 3))
    
    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(230, 20, as_cmap=True)
    
    sns.heatmap(_cm, annot=True, cmap=cmap, xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.show()

In [13]:
units = [128]
activations = ['relu', 'tanh', 'sigmoid']
kernel_size = [3, 5]
learning_rate = [0.001, 0.0001, 0.00001]
optimizer = ['Adam', 'RMSprop', 'Nadam']
dropout = [0.1, 0.2, 0.3]

# Model

In [14]:
class Baseline():
    def __init__(self, _input_dim, unit, activation, kernel_size, learning_rate, optimizer, dropout, metrics=METRICS):
        # Define the model architecture
        model = Sequential()
        
        # hidden layer
        model.add(Dense(unit, activation=activation, input_dim=_input_dim))
        model.add(Dropout(dropout))
        
        # hidden layer
        model.add(Dense(unit, activation=activation))
        model.add(Dropout(dropout))
        
        # hidden layer
        model.add(Dense(unit, activation=activation))
        model.add(Dropout(dropout))
        
        # hidden layer
        model.add(Dense(unit, activation=activation))
        model.add(Dropout(dropout))
        
        # output layer
        model.add(Dense(1, activation='sigmoid'))

        if optimizer == 'Adam':
            ops = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        elif optimizer == 'RMSprop':
            ops = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
        elif optimizer == 'Nadam':
            ops = tf.keras.optimizers.Nadam(learning_rate=learning_rate)
            
        # Compile the model
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        self.model = model
        
    def train(self, X_train, y_train, X_val, y_val, epochs, batch_size):
        history = self.model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, callbacks=[early_stopping], verbose=2)
        return history

    def evaluate(self, X_test, y_test):
        results = self.model.evaluate(X_test, y_test, verbose=0)
        return results

    def predict(self, X): #X_train X_test
        predictions = self.model.predict(X)
        return predictions
    
    def summary(self):
        self.model.summary()
        
    def metrics_names(self):
        return self.model.metrics_names

# Model MinMax

In [15]:
df = df_damage_min_max.drop(columns=columns)

X = df.drop(columns=['Damage'])
y = df['Damage']

# split the data into a holdout set and the rest for K-Fold cross-validation
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, train_size=training_percent, random_state=42)

In [16]:
# Initialize StandardScaler
scaler = StandardScaler()

# Fit scaler on training data
scaler.fit(X_train)

# Transform both training and test data
X_train_scaled = scaler.transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

In [17]:
best_acc = 0.0
baseline_values = { 'units': 0, 'activation': '', 'kernel_size': 0, 'learning_rate' : 0.0, 'optimizer': '', 'dropout': 0.0 }

In [18]:
input_dim = X_train.shape[1]

In [19]:
for u in units:
    for a in activations:
        for k in kernel_size:
            for l in learning_rate:
                for o in optimizer:
                    for d in dropout:
                        print('\nUnits:', u, 'Activation:', a, 'Kernel size:', k, 'Learning rate:', l, 'Optimizer:', o, 'Dropout:', d)
                        for train_index, val_index in kf.split(X_train_scaled):
                            # split the dataset into training and validation sets for this fold
                            X_train_kf, X_val_kf = X_train_scaled[train_index], X_train_scaled[val_index]
                            y_train_kf, y_val_kf = y_train.iloc[train_index], y_train.iloc[val_index]

                            model = Baseline(input_dim, u, a, k, l, o, d)

                            history = model.train(X_train_kf, y_train_kf, X_val_kf, y_val_kf, epochs, batch_size)
                            if (history.history['val_accuracy'][-1] > best_acc):
                                best_acc = history.history['val_accuracy'][-1]
                                baseline_values = { 'units': u, 'activation': a, 'kernel_size': k, 'learning_rate' : l, 'optimizer': o, 'dropout': d }


Units: 128 Activation: relu Kernel size: 3 Learning rate: 0.001 Optimizer: Adam Dropout: 0.1
Epoch 1/50
3/3 - 1s - loss: 0.6181 - accuracy: 0.8409 - val_loss: 0.5121 - val_accuracy: 0.9111 - 1s/epoch - 477ms/step
Epoch 2/50
3/3 - 0s - loss: 0.4532 - accuracy: 0.9318 - val_loss: 0.3964 - val_accuracy: 0.9111 - 41ms/epoch - 14ms/step
Epoch 3/50
3/3 - 0s - loss: 0.3581 - accuracy: 0.8864 - val_loss: 0.3234 - val_accuracy: 0.9111 - 42ms/epoch - 14ms/step
Epoch 4/50
3/3 - 0s - loss: 0.2699 - accuracy: 0.9091 - val_loss: 0.2960 - val_accuracy: 0.9111 - 36ms/epoch - 12ms/step
Epoch 5/50
3/3 - 0s - loss: 0.2337 - accuracy: 0.9091 - val_loss: 0.3022 - val_accuracy: 0.9111 - 34ms/epoch - 11ms/step
Epoch 6/50
3/3 - 0s - loss: 0.2240 - accuracy: 0.9091 - val_loss: 0.3248 - val_accuracy: 0.9111 - 33ms/epoch - 11ms/step
Epoch 7/50
3/3 - 0s - loss: 0.2416 - accuracy: 0.9318 - val_loss: 0.3499 - val_accuracy: 0.9111 - 34ms/epoch - 11ms/step
Epoch 8/50
3/3 - 0s - loss: 0.2219 - accuracy: 0.9318 - val_

Epoch 3/50
3/3 - 0s - loss: 0.4918 - accuracy: 0.8764 - val_loss: 0.4576 - val_accuracy: 0.8409 - 46ms/epoch - 15ms/step
Epoch 4/50
3/3 - 0s - loss: 0.3920 - accuracy: 0.9213 - val_loss: 0.3881 - val_accuracy: 0.8636 - 42ms/epoch - 14ms/step
Epoch 5/50
3/3 - 0s - loss: 0.3018 - accuracy: 0.9326 - val_loss: 0.3461 - val_accuracy: 0.8636 - 34ms/epoch - 11ms/step
Epoch 6/50
3/3 - 0s - loss: 0.2606 - accuracy: 0.9213 - val_loss: 0.3362 - val_accuracy: 0.8636 - 33ms/epoch - 11ms/step
Epoch 7/50
3/3 - 0s - loss: 0.2399 - accuracy: 0.9213 - val_loss: 0.3480 - val_accuracy: 0.8636 - 34ms/epoch - 11ms/step
Epoch 8/50
3/3 - 0s - loss: 0.2257 - accuracy: 0.9326 - val_loss: 0.3665 - val_accuracy: 0.8636 - 33ms/epoch - 11ms/step
Epoch 9/50
3/3 - 0s - loss: 0.2210 - accuracy: 0.9438 - val_loss: 0.3759 - val_accuracy: 0.8636 - 33ms/epoch - 11ms/step
Epoch 10/50
3/3 - 0s - loss: 0.2288 - accuracy: 0.9438 - val_loss: 0.3819 - val_accuracy: 0.8636 - 32ms/epoch - 11ms/step
Epoch 11/50
3/3 - 0s - loss: 0.

Epoch 2/50
3/3 - 0s - loss: 0.5735 - accuracy: 0.7978 - val_loss: 0.5238 - val_accuracy: 0.7955 - 35ms/epoch - 12ms/step
Epoch 3/50
3/3 - 0s - loss: 0.4937 - accuracy: 0.8876 - val_loss: 0.4611 - val_accuracy: 0.8182 - 35ms/epoch - 12ms/step
Epoch 4/50
3/3 - 0s - loss: 0.4109 - accuracy: 0.9213 - val_loss: 0.4038 - val_accuracy: 0.8636 - 35ms/epoch - 12ms/step
Epoch 5/50
3/3 - 0s - loss: 0.3420 - accuracy: 0.9326 - val_loss: 0.3587 - val_accuracy: 0.8636 - 36ms/epoch - 12ms/step
Epoch 6/50
3/3 - 0s - loss: 0.2679 - accuracy: 0.9213 - val_loss: 0.3352 - val_accuracy: 0.8636 - 38ms/epoch - 13ms/step
Epoch 7/50
3/3 - 0s - loss: 0.2135 - accuracy: 0.9551 - val_loss: 0.3368 - val_accuracy: 0.8636 - 57ms/epoch - 19ms/step
Epoch 8/50
3/3 - 0s - loss: 0.2286 - accuracy: 0.9438 - val_loss: 0.3530 - val_accuracy: 0.8409 - 51ms/epoch - 17ms/step
Epoch 9/50
3/3 - 0s - loss: 0.2153 - accuracy: 0.9213 - val_loss: 0.3727 - val_accuracy: 0.8409 - 36ms/epoch - 12ms/step
Epoch 10/50
3/3 - 0s - loss: 0.2

Epoch 3/50
3/3 - 0s - loss: 0.2346 - accuracy: 0.9213 - val_loss: 0.3658 - val_accuracy: 0.8636 - 126ms/epoch - 42ms/step
Epoch 4/50
3/3 - 0s - loss: 0.2125 - accuracy: 0.9213 - val_loss: 0.3877 - val_accuracy: 0.8636 - 60ms/epoch - 20ms/step
Epoch 5/50
3/3 - 0s - loss: 0.2142 - accuracy: 0.9438 - val_loss: 0.3767 - val_accuracy: 0.8636 - 50ms/epoch - 17ms/step
Epoch 6/50
3/3 - 0s - loss: 0.1832 - accuracy: 0.9438 - val_loss: 0.4105 - val_accuracy: 0.8636 - 154ms/epoch - 51ms/step
Epoch 7/50
3/3 - 0s - loss: 0.1916 - accuracy: 0.9551 - val_loss: 0.4228 - val_accuracy: 0.8864 - 77ms/epoch - 26ms/step
Epoch 8/50
3/3 - 0s - loss: 0.1519 - accuracy: 0.9663 - val_loss: 0.4096 - val_accuracy: 0.8636 - 144ms/epoch - 48ms/step
Epoch 9/50
3/3 - 0s - loss: 0.1197 - accuracy: 0.9551 - val_loss: 0.4536 - val_accuracy: 0.8864 - 54ms/epoch - 18ms/step
Epoch 10/50
3/3 - 0s - loss: 0.1056 - accuracy: 0.9663 - val_loss: 0.4847 - val_accuracy: 0.8864 - 110ms/epoch - 37ms/step
Epoch 11/50
3/3 - 0s - loss

Epoch 7/50
3/3 - 0s - loss: 0.2784 - accuracy: 0.9213 - val_loss: 0.1440 - val_accuracy: 0.9091 - 50ms/epoch - 17ms/step
Epoch 8/50
3/3 - 0s - loss: 0.2498 - accuracy: 0.9213 - val_loss: 0.1571 - val_accuracy: 0.9091 - 44ms/epoch - 15ms/step
Epoch 9/50
3/3 - 0s - loss: 0.2318 - accuracy: 0.9213 - val_loss: 0.1597 - val_accuracy: 0.9091 - 50ms/epoch - 17ms/step
Epoch 10/50
3/3 - 0s - loss: 0.2030 - accuracy: 0.9551 - val_loss: 0.1588 - val_accuracy: 0.9091 - 43ms/epoch - 14ms/step
Epoch 11/50
3/3 - 0s - loss: 0.2308 - accuracy: 0.9326 - val_loss: 0.1689 - val_accuracy: 0.9091 - 42ms/epoch - 14ms/step
Epoch 12/50
3/3 - 0s - loss: 0.2233 - accuracy: 0.9326 - val_loss: 0.1387 - val_accuracy: 0.9091 - 42ms/epoch - 14ms/step
Epoch 13/50
3/3 - 0s - loss: 0.1679 - accuracy: 0.9663 - val_loss: 0.1373 - val_accuracy: 0.9091 - 41ms/epoch - 14ms/step
Epoch 14/50
3/3 - 0s - loss: 0.1886 - accuracy: 0.9438 - val_loss: 0.1295 - val_accuracy: 0.9318 - 44ms/epoch - 15ms/step
Epoch 15/50
3/3 - 0s - loss

...

Epoch 5/50
3/3 - 0s - loss: 0.6375 - accuracy: 0.6404 - val_loss: 0.6594 - val_accuracy: 0.6136 - 38ms/epoch - 13ms/step
Epoch 6/50
3/3 - 0s - loss: 0.6418 - accuracy: 0.6629 - val_loss: 0.6565 - val_accuracy: 0.6136 - 39ms/epoch - 13ms/step
Epoch 7/50
3/3 - 0s - loss: 0.6350 - accuracy: 0.6517 - val_loss: 0.6517 - val_accuracy: 0.6136 - 39ms/epoch - 13ms/step
Epoch 8/50
3/3 - 0s - loss: 0.6165 - accuracy: 0.6517 - val_loss: 0.6484 - val_accuracy: 0.6136 - 36ms/epoch - 12ms/step
Epoch 9/50
3/3 - 0s - loss: 0.6296 - accuracy: 0.6517 - val_loss: 0.6415 - val_accuracy: 0.6136 - 34ms/epoch - 11ms/step
Epoch 10/50
3/3 - 0s - loss: 0.6201 - accuracy: 0.6517 - val_loss: 0.6302 - val_accuracy: 0.6136 - 37ms/epoch - 12ms/step
Epoch 11/50
3/3 - 0s - loss: 0.6194 - accuracy: 0.6517 - val_loss: 0.6140 - val_accuracy: 0.6136 - 34ms/epoch - 11ms/step
Epoch 12/50
3/3 - 0s - loss: 0.5816 - accuracy: 0.6742 - val_loss: 0.5951 - val_accuracy: 0.6136 - 34ms/epoch - 11ms/step
Epoch 13/50
3/3 - 0s - loss: 

Epoch 40/50
3/3 - 0s - loss: 0.2840 - accuracy: 0.9101 - val_loss: 0.1573 - val_accuracy: 0.9318 - 33ms/epoch - 11ms/step
Epoch 41/50
3/3 - 0s - loss: 0.2881 - accuracy: 0.8989 - val_loss: 0.1596 - val_accuracy: 0.9318 - 32ms/epoch - 11ms/step
Epoch 42/50
3/3 - 0s - loss: 0.2534 - accuracy: 0.9213 - val_loss: 0.1588 - val_accuracy: 0.9318 - 32ms/epoch - 11ms/step
Epoch 43/50
3/3 - 0s - loss: 0.2764 - accuracy: 0.9213 - val_loss: 0.1556 - val_accuracy: 0.9318 - 32ms/epoch - 11ms/step
Epoch 44/50
3/3 - 0s - loss: 0.2849 - accuracy: 0.9101 - val_loss: 0.1442 - val_accuracy: 0.9318 - 33ms/epoch - 11ms/step
Epoch 45/50
3/3 - 0s - loss: 0.2760 - accuracy: 0.9213 - val_loss: 0.1449 - val_accuracy: 0.9318 - 31ms/epoch - 10ms/step
Epoch 46/50
3/3 - 0s - loss: 0.2738 - accuracy: 0.9101 - val_loss: 0.1503 - val_accuracy: 0.9318 - 33ms/epoch - 11ms/step
Epoch 47/50
3/3 - 0s - loss: 0.2687 - accuracy: 0.8989 - val_loss: 0.1442 - val_accuracy: 0.9318 - 33ms/epoch - 11ms/step
Epoch 48/50
3/3 - 0s - l

Epoch 15/50
3/3 - 0s - loss: 0.5832 - accuracy: 0.6517 - val_loss: 0.5751 - val_accuracy: 0.6136 - 41ms/epoch - 14ms/step
Epoch 16/50
3/3 - 0s - loss: 0.5250 - accuracy: 0.7191 - val_loss: 0.5470 - val_accuracy: 0.7500 - 36ms/epoch - 12ms/step
Epoch 17/50
3/3 - 0s - loss: 0.4702 - accuracy: 0.8427 - val_loss: 0.5157 - val_accuracy: 0.7727 - 35ms/epoch - 12ms/step
Epoch 18/50
3/3 - 0s - loss: 0.4611 - accuracy: 0.8090 - val_loss: 0.4804 - val_accuracy: 0.7727 - 42ms/epoch - 14ms/step
Epoch 19/50
3/3 - 0s - loss: 0.3977 - accuracy: 0.8652 - val_loss: 0.4485 - val_accuracy: 0.8182 - 41ms/epoch - 14ms/step
Epoch 20/50
3/3 - 0s - loss: 0.4044 - accuracy: 0.8876 - val_loss: 0.4195 - val_accuracy: 0.8182 - 38ms/epoch - 13ms/step
Epoch 21/50
3/3 - 0s - loss: 0.3205 - accuracy: 0.8989 - val_loss: 0.3957 - val_accuracy: 0.8182 - 36ms/epoch - 12ms/step
Epoch 22/50
3/3 - 0s - loss: 0.3006 - accuracy: 0.9326 - val_loss: 0.3768 - val_accuracy: 0.8182 - 38ms/epoch - 13ms/step
Epoch 23/50
3/3 - 0s - l

Epoch 44/50
3/3 - 0s - loss: 0.2876 - accuracy: 0.8876 - val_loss: 0.1553 - val_accuracy: 0.9318 - 35ms/epoch - 12ms/step
Epoch 45/50
3/3 - 0s - loss: 0.2631 - accuracy: 0.9101 - val_loss: 0.1600 - val_accuracy: 0.9318 - 35ms/epoch - 12ms/step
Epoch 46/50
3/3 - 0s - loss: 0.2861 - accuracy: 0.8989 - val_loss: 0.1709 - val_accuracy: 0.9318 - 34ms/epoch - 11ms/step
Epoch 47/50
3/3 - 0s - loss: 0.2868 - accuracy: 0.9101 - val_loss: 0.1741 - val_accuracy: 0.9091 - 34ms/epoch - 11ms/step
Epoch 48/50
3/3 - 0s - loss: 0.2854 - accuracy: 0.9101 - val_loss: 0.1617 - val_accuracy: 0.9318 - 34ms/epoch - 11ms/step
Epoch 49/50
3/3 - 0s - loss: 0.2901 - accuracy: 0.8989 - val_loss: 0.1521 - val_accuracy: 0.9318 - 34ms/epoch - 11ms/step
Epoch 50/50
3/3 - 0s - loss: 0.2746 - accuracy: 0.9213 - val_loss: 0.1506 - val_accuracy: 0.9318 - 33ms/epoch - 11ms/step

Units: 128 Activation: sigmoid Kernel size: 5 Learning rate: 1e-05 Optimizer: Nadam Dropout: 0.3
Epoch 1/50
3/3 - 2s - loss: 0.7650 - accuracy: 0

Epoch 7/50
3/3 - 0s - loss: 0.7002 - accuracy: 0.5169 - val_loss: 0.6782 - val_accuracy: 0.9091 - 33ms/epoch - 11ms/step
Epoch 8/50
3/3 - 0s - loss: 0.7126 - accuracy: 0.5506 - val_loss: 0.6709 - val_accuracy: 0.8636 - 32ms/epoch - 11ms/step
Epoch 9/50
3/3 - 0s - loss: 0.6807 - accuracy: 0.5506 - val_loss: 0.6573 - val_accuracy: 0.7273 - 32ms/epoch - 11ms/step
Epoch 10/50
3/3 - 0s - loss: 0.6723 - accuracy: 0.5730 - val_loss: 0.6430 - val_accuracy: 0.7273 - 32ms/epoch - 11ms/step
Epoch 11/50
3/3 - 0s - loss: 0.7586 - accuracy: 0.4270 - val_loss: 0.6216 - val_accuracy: 0.7273 - 33ms/epoch - 11ms/step
Epoch 12/50
3/3 - 0s - loss: 0.6626 - accuracy: 0.5618 - val_loss: 0.6066 - val_accuracy: 0.7273 - 33ms/epoch - 11ms/step
Epoch 13/50
3/3 - 0s - loss: 0.6382 - accuracy: 0.6629 - val_loss: 0.5956 - val_accuracy: 0.7273 - 34ms/epoch - 11ms/step


In [20]:
print(best_acc)
print(baseline_values)

1.0
{'units': 128, 'activation': 'tanh', 'kernel_size': 5, 'learning_rate': 1e-05, 'optimizer': 'RMSprop', 'dropout': 0.1}


In [21]:
#STOP